In [12]:
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI


In [4]:
df = pd.read_csv('./data/postdefined_users_gpt.csv', encoding = "ISO-8859-1")
df

,category,rang,full_name,birth_year,location,run_link,run_year,finish_time,age_year,age_group,interesr_rate,payment_monthly,draw_average,late_fee,referral,income
0,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------
1,42-M30,84,Jimenez Pineda Enrique,1981,E-Soria,https://services.datasport.com/2014/lauf/zueri...,2014,3:03:35,33,31-40,0.03,200,5000,50,Y,30000
2,42-M30,477,Jacot-Descombes Sebast,1978,Wavre,https://services.datasport.com/2016/lauf/zueri...,2016,4:10:39,38,31-40,0.05,500,20000,0,N,30000
3,42-M45,305,Jallas Amigo JosÃ© Gust,1970,E-Palma de Mallorca,https://services.datasport.com/2016/lauf/zueri...,2016,4:18:58,46,41-50,0.05,500,20000,0,N,30000
4,42-WElit,6,Jemutai Bartenge Prisc,1986,Kenia,https://services.datasport.com/2014/lauf/zueri...,2014,2:41:59,28,21-30,0.05,500,20000,0,N,30000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29721,42-M70,12,RÃ¼egg Walter,1936,ZÃ¼rich,https://services.datasport.com/2016/lauf/zueri...,2016,4:57:40,80,>50,0.016,300,2500,20,N,80000
29722,42-M70,17,Oehninger Ernst,1936,Watt,https://services.datasport.com/2016/lauf/zueri...,2016,1:23:12,80,>50,0.016,300,2500,20,N,80000
29723,10-M70,14,Oehninger Ernst,1936,Watt,https://services.datasport.com/2018/lauf/zueri...,2018,1:10:24,82,>50,0.016,300,2500,20,N,80000
29724,10-M70,5,Frenchy Remeomo,1935,ZÃ¼rich,https://services.datasport.com/2015/lauf/zueri...,2015,0:53:34,80,>50,0.016,300,2500,20,N,80000


In [5]:
pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()

    return s

df['category']= df["category"].apply(lambda x : normalize_text(x))

In [6]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["category"].apply(lambda x: len(tokenizer.encode(x)))
df = df[df.n_tokens<8192]
len(df)

29726

In [7]:
df

,category,rang,full_name,birth_year,location,run_link,run_year,finish_time,age_year,age_group,interesr_rate,payment_monthly,draw_average,late_fee,referral,income,n_tokens
0,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,-----------,1
1,42-M30,84,Jimenez Pineda Enrique,1981,E-Soria,https://services.datasport.com/2014/lauf/zueri...,2014,3:03:35,33,31-40,0.03,200,5000,50,Y,30000,3
2,42-M30,477,Jacot-Descombes Sebast,1978,Wavre,https://services.datasport.com/2016/lauf/zueri...,2016,4:10:39,38,31-40,0.05,500,20000,0,N,30000,3
3,42-M45,305,Jallas Amigo JosÃ© Gust,1970,E-Palma de Mallorca,https://services.datasport.com/2016/lauf/zueri...,2016,4:18:58,46,41-50,0.05,500,20000,0,N,30000,3
4,42-WElit,6,Jemutai Bartenge Prisc,1986,Kenia,https://services.datasport.com/2014/lauf/zueri...,2014,2:41:59,28,21-30,0.05,500,20000,0,N,30000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29721,42-M70,12,RÃ¼egg Walter,1936,ZÃ¼rich,https://services.datasport.com/2016/lauf/zueri...,2016,4:57:40,80,>50,0.016,300,2500,20,N,80000,3
29722,42-M70,17,Oehninger Ernst,1936,Watt,https://services.datasport.com/2016/lauf/zueri...,2016,1:23:12,80,>50,0.016,300,2500,20,N,80000,3
29723,10-M70,14,Oehninger Ernst,1936,Watt,https://services.datasport.com/2018/lauf/zueri...,2018,1:10:24,82,>50,0.016,300,2500,20,N,80000,3
29724,10-M70,5,Frenchy Remeomo,1935,ZÃ¼rich,https://services.datasport.com/2015/lauf/zueri...,2015,0:53:34,80,>50,0.016,300,2500,20,N,80000,3


In [8]:
sample_encode = tokenizer.encode(df.category[0])
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

[b'-----------']

In [9]:
len(decode)

1

In [23]:
client = AzureOpenAI(
  azure_deployment = "",
  api_key = "",
  api_version = "2024-06-01",
  azure_endpoint = ""
)


def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df['ada_v2'] = df["category"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))

In [24]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(df, user_query, top_n=4, to_print=True):
    embedding = get_embedding(
        user_query,
        model="text-embedding-ada-002" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df, "Can I get information on the different type of categories?", top_n=4)

,category,rang,full_name,birth_year,location,run_link,run_year,finish_time,age_year,age_group,interesr_rate,payment_monthly,draw_average,late_fee,referral,income,n_tokens,ada_v2,similarities
6798,10-M30,162,Giger Philipp,1986,Galgenen,https://services.datasport.com/2016/lauf/zueri...,2016,0:48:46,30,21-30,0.035,500,20000,25,Y,250000,3,"[-0.01654931530356407, -0.0361999087035656, 0....",0.757122
9766,10-M30,275,Lepoittevin GrÃ©goire,1983,ZÃ¼rich,https://services.datasport.com/2018/lauf/zueri...,2018,0:52:24,35,31-40,0.035,500,20000,50,Y,250000,3,"[-0.01654931530356407, -0.0361999087035656, 0....",0.757122
9702,10-M30,275,Swerissen Mark,1983,ZÃ¼rich,https://services.datasport.com/2016/lauf/zueri...,2016,0:55:30,33,31-40,0.035,500,20000,50,Y,250000,3,"[-0.01654931530356407, -0.0361999087035656, 0....",0.757122
9703,10-M30,357,Stoop Patrick,1983,Russikon,https://services.datasport.com/2018/lauf/zueri...,2018,0:55:29,35,31-40,0.035,500,20000,50,Y,250000,3,"[-0.01654931530356407, -0.0361999087035656, 0....",0.757122


In [25]:
res["category"][6798]

'10-M30'